In [101]:
import pandas as pd

In [102]:
# CSV 파일 경로
csv_file_path = r"C:/Users/yeeun/Desktop/mini_project/서울시 음향신호기 관련 정보.csv"

# CSV 파일을 데이터프레임으로 읽어오기
df = pd.read_csv(csv_file_path, encoding='cp949')

In [103]:
df

,음향신호관리번호,지주관리번호,방향 (공통),설치일,교체일,X좌표,Y좌표,제조회사,시설번호,작업구분 (공통),표출구분 (공통),종류,신규정규화ID,상태 (공통),공사관리번호,음향신호관리번호.1,이력ID,위치정보,공사형태 (공통)
0,24-0000019810,02-0000220951,NaN,20221121,20221121,202410.75432,540397.01740,(주)한길에이치씨,NaN,1,2,1,4057421,1.0,2022-0101-101,24-019810,24538,NaN,001
1,24-0000010308,02-0000087079,180.0,20000101,20000101,203867.87114,563343.20795,NaN,NaN,1,2,1,4583314,1.0,NaN,24-010308,15036,1.0,001
2,24-0000012524,02-0000081308,0.0,NaN,NaN,207048.17500,553518.03124,NaN,NaN,1,2,1,5343753,1.0,NaN,24-012524,17252,1.0,001
3,24-0000009796,02-0000118897,180.0,20131207,20131207,207771.50524,557685.44648,(주)한길에이치씨,NaN,1,2,1,5461184,1.0,2013-1801-007,24-009796,14524,1.0,001
4,24-0000021536,02-0000180500,NaN,20230831,20230831,NaN,NaN,대한신호,NaN,1,2,1,1270382,1.0,2023-0101-013,24-021536,26264,NaN,001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26632,24-0000021859,02-0000223738,NaN,20231103,20231103,NaN,NaN,대한신호,NaN,1,2,1,5404318,1.0,2023-0101-133,24-021859,26587,NaN,001
26633,24-0000021858,02-0000223736,NaN,20231103,20231103,NaN,NaN,대한신호,NaN,1,2,1,5404316,1.0,2023-0101-133,24-021858,26586,NaN,001
26634,24-0000017646,02-0000082889,NaN,20201202,20201202,204845.88245,558851.71294,대한신호,NaN,1,2,1,5404315,1.0,2023-0101-133,24-017646,22374,NaN,004
26635,24-0000021857,02-0000090709,NaN,20231113,20231113,NaN,NaN,대한신호,NaN,1,2,1,5513301,1.0,2023-0101-133,24-021857,26585,NaN,001


In [104]:
df_voice_signal = df[['음향신호관리번호', 'X좌표', 'Y좌표']]
df_voice_signal

,음향신호관리번호,X좌표,Y좌표
0,24-0000019810,202410.75432,540397.01740
1,24-0000010308,203867.87114,563343.20795
2,24-0000012524,207048.17500,553518.03124
3,24-0000009796,207771.50524,557685.44648
4,24-0000021536,NaN,NaN
...,...,...,...
26632,24-0000021859,NaN,NaN
26633,24-0000021858,NaN,NaN
26634,24-0000017646,204845.88245,558851.71294
26635,24-0000021857,NaN,NaN


In [105]:
df_voice_signal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26637 entries, 0 to 26636
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   음향신호관리번호  26637 non-null  object 
 1   X좌표       24614 non-null  float64
 2   Y좌표       24614 non-null  float64
dtypes: float64(2), object(1)
memory usage: 624.4+ KB


복지시설 300m 반경 개수 세기

In [106]:
import geopandas as gpd
from shapely.geometry import Point


In [107]:
csv_file_path = r"C:/Users/yeeun/Desktop/mini_project/geocode_welfare_transformed.csv"
df_welfare = pd.read_csv(csv_file_path)

In [108]:
# GeoDataFrame으로 변환
gdf_welfare = gpd.GeoDataFrame(df_welfare, geometry=gpd.points_from_xy(df_welfare['X'], df_welfare['Y']))
gdf_voice_signal = gpd.GeoDataFrame(df_voice_signal, geometry=gpd.points_from_xy(df_voice_signal['X좌표'], df_voice_signal['Y좌표']))

# 결측값이 있는 행 제거 및 인덱스 재설정
gdf_welfare = gdf_welfare[~gdf_welfare[['X', 'Y']].isna().sum(axis=1).astype(bool)].reset_index(drop=True)
gdf_voice_signal = gdf_voice_signal[~gdf_voice_signal[['X좌표', 'Y좌표']].isna().sum(axis=1).astype(bool)].reset_index(drop=True)

In [109]:
buffer_radius = 300  # 반경 300m

# 장애인복지시설의 버퍼 생성
gdf_welfare['buffer'] = gdf_welfare.geometry.buffer(buffer_radius)

# 각 장애인복지시설의 버퍼에 포함된 안전표지판 개수 계산
def count_voice_signals(row):
    try:
        return gdf_voice_signal[row['buffer'].contains(gdf_voice_signal.unary_union)].shape[0]
    except KeyError:
        return 0

gdf_welfare['voice_signal_count'] = gdf_welfare.apply(count_voice_signals, axis=1)

# 필요한 열만 선택
df_voice_signal_count = gdf_welfare[['시설명', 'voice_signal_count']]

KeyboardInterrupt: 

In [ ]:
df_voice_signal_count